In [4]:
import numpy as np
from scipy import misc as misc
import tensorflow as tf
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
tf.set_random_seed(777)
tmp = np.load('/data1/jerry/face/Network/data.npz')
train_img = tmp['train_img']
train_lab = tmp['train_lab']

In [7]:
train_img2 = None
for i, timg in enumerate(train_img):
    timg2 = np.reshape(misc.imresize(timg, (224,224,3)), (1,224,224,3))
    if i == 0:
        train_img2 = timg2
    else:
        train_img2 = np.concatenate((train_img2, timg2), axis=0)

In [8]:
num_classes = 2
X = tf.placeholder(tf.float32, shape=[None, 224,224,3])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])
keep = tf.placeholder(tf.float32)

In [9]:
weights = {
    'wc1' : tf.get_variable('wc1', [7, 7, 3, 96], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),    # 240
    'wc2' : tf.get_variable('wc2', [5, 5, 96, 256], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),   # 120
    'wc3' : tf.get_variable('wc3', [3, 3, 256, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),   # 60
    'wc4' : tf.get_variable('wc4', [3, 3, 512, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),  # 30
    'wc5' : tf.get_variable('wc5', [3, 3, 512, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()), # 15
    'wd1' : tf.get_variable('wd1', [13*13*512, 4048], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),
    'wd2' : tf.get_variable('wd2', [4048, 4049], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),
    'wd3' : tf.get_variable('wd3', [4049, num_classes], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer())
}

biases = {
    'bc1' : tf.Variable(tf.zeros([96]), trainable=False, dtype=tf.float32),
    'bc2' : tf.Variable(tf.zeros([256]), trainable=False, dtype=tf.float32),
    'bc3' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bc4' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bc5' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bd1' : tf.Variable(tf.zeros([4048]), trainable=False, dtype=tf.float32),
    'bd2' : tf.Variable(tf.zeros([4049]), trainable=False, dtype=tf.float32),
    'bd3' : tf.Variable(tf.zeros([num_classes]), trainable=False, dtype=tf.float32)
}

In [10]:
def model(x, w, b, k):
    
    conv1 = tf.nn.conv2d(x, w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
    act1 = tf.nn.relu(conv1 + b['bc1'])
    pool1 = tf.nn.max_pool(act1, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='SAME')
    print(pool1)
    conv2 = tf.nn.conv2d(pool1, w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
    act2 = tf.nn.relu(conv2 + b['bc2'])
    pool2 = tf.nn.max_pool(act2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print(pool2)
    conv3 = tf.nn.conv2d(pool2, w['wc3'], strides=[1, 1, 1, 1], padding='SAME')
    act3 = tf.nn.relu(conv3 + b['bc3'])
    
    conv4 = tf.nn.conv2d(conv3, w['wc4'], strides=[1, 1, 1, 1], padding='SAME')
    act4 = tf.nn.relu(conv4 + b['bc4'])
    
    conv5 = tf.nn.conv2d(conv4, w['wc5'], strides=[1, 1, 1, 1], padding='SAME')
    act5 = tf.nn.relu(conv5 + b['bc5'])
    pool5 = tf.nn.max_pool(act5, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='SAME')
    print(pool5)
    flat = tf.reshape(pool5, [-1, 13*13*512])
    
    act6 = tf.nn.relu(tf.matmul(flat, w['wd1'])+b['bd1'])
    drop1 = tf.nn.dropout(act6, keep_prob=k)
    
    act7 = tf.nn.relu(tf.matmul(drop1, w['wd2'])+b['bd2'])
    drop2 = tf.nn.dropout(act7, keep_prob=k)
    
    output = tf.matmul(drop2, w['wd3'])+b['bd3']
    
    out = {
        'act1':act1,
        'act2':act2,
        'act3':act3,
        'act4':act4,
        'act5':act5,
        'act6':act6,
        'act7':act7,
        'output':output
        
    }
    
    
    return out

In [11]:
pred = model(X, weights, biases, keep)

Tensor("MaxPool:0", shape=(?, 75, 75, 96), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 38, 38, 256), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 13, 13, 512), dtype=float32)


In [8]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred['output'], labels=Y))

train = tf.train.AdamOptimizer(0.0001).minimize(cost)

collect = tf.equal(tf.argmax(pred['output'], axis=1), tf.argmax(Y, axis=1))

accuracy = tf.reduce_mean(tf.cast(collect, tf.float32))

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [10]:
epochs = 10
batch = 5



for epoch in range(epochs):
    
    total_batch = int(len(train_img2)/batch)
    
    print("\n{0}/{1}".format(epoch+1, epochs))
    
    for i in range(total_batch):
        
        batch_idx = np.random.choice(len(train_img2), batch, replace=False)
        
        batch_x = train_img2[batch_idx]
        
        batch_y = train_lab[batch_idx]
        
        sess.run(train, feed_dict={X:batch_x, Y:batch_y, keep:0.7})
        
        if i%10 ==0:
            
            test_idx = np.random.choice(len(train_img2), int(len(train_img2)/2), replace=False)
            
            print("Cost : ", sess.run(cost, feed_dict={X:train_img2[test_idx], Y:train_lab[test_idx], keep:1.}))
            
            print("Accuracy : ", sess.run(accuracy, feed_dict={X:train_img2[test_idx], Y:train_lab[test_idx], keep:1.}))
            
#saver.save(sess, './checkpoint/')        
print("\n Training Done!")


1/10


ResourceExhaustedError: OOM when allocating tensor with shape[116,96,224,224]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_Placeholder_0_0/_19, wc1/read)]]

Caused by op 'Conv2D', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).